In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
movie_data = pd.read_csv('./movie_data/movies_metadata.csv')
movie_data =  movie_data.loc[movie_data['original_language'] == 'en', :]
movie_data = movie_data[['id', 'title', 'original_language', 'genres']]

print(movie_data.shape)
movie_data.head()

(32269, 4)


C:\Users\user\AppData\Local\Temp\ipykernel_30828\1894366887.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_data = pd.read_csv('./movie_data/movies_metadata.csv')


,id,title,original_language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"


In [43]:
movie_keyword = pd.read_csv('./movie_data/keywords.csv')
print(movie_keyword.shape)
movie_keyword.head()

(46419, 2)


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [44]:
movie_data.id = movie_data.id.astype(int)
movie_keyword.id = movie_keyword.id.astype(int)
movie_data = pd.merge(movie_data, movie_keyword, on='id')
print(movie_data.shape)
movie_data.head()

(32852, 5)


,id,title,original_language,genres,keywords
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [45]:
# list(dict()) 안에 문자열 형태가 들어가있는 경우
movie_data['genres'] = movie_data['genres'].apply(literal_eval)
movie_data['genres'] = movie_data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x: " ".join(x))

movie_data['keywords'] = movie_data['keywords'].apply(literal_eval)
movie_data['keywords'] = movie_data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [46]:
movie_data.head()

,id,title,original_language,genres,keywords
0,862,Toy Story,en,Animation Comedy Family,jealousy toy boy friendship friends rivalry bo...
1,8844,Jumanji,en,Adventure Fantasy Family,board game disappearance based on children's b...
2,15602,Grumpier Old Men,en,Romance Comedy,fishing best friend duringcreditsstinger old men
3,31357,Waiting to Exhale,en,Comedy Drama Romance,based on novel interracial relationship single...
4,11862,Father of the Bride Part II,en,Comedy,baby midlife crisis confidence aging daughter ...


## TF-IDF 벡터화

In [47]:
tfidf_vector = TfidfVectorizer()
#tfidf_vector = TfidfVectorizer(ngram_range=(1,2))
tfidf_matrix = tfidf_vector.fit_transform(movie_data['genres'] + " " + movie_data['keywords']).toarray()
#tfidf_matrix = tfidf_vector.fit_transform(movie_data['genres']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names()

c:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [48]:
tfidf_matrix.shape

(32852, 11437)

In [49]:
type(tfidf_matrix)

numpy.ndarray

In [50]:
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index=movie_data.title)
#print(tfidf_matrix)
tfidf_matrix.head()

,077,10,11,13,1500s,15th,16th,17th,1812,18th,...,βάφτηκε,γη,κόκκινο,το,χώμα,миньоны,卧底肥妈,绝地奶霸,自然界大事件,超级妈妈
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jumanji,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grumpier Old Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Waiting to Exhale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Father of the Bride Part II,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 유사도 구하기 (코사인 유사도)

In [53]:
%%time
cosine_sim = cosine_similarity(tfidf_matrix)

Wall time: 41.9 s


In [54]:
cosine_sim.shape

(32852, 32852)

In [55]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=movie_data.title, columns=movie_data.title)
print(cosine_sim_df.shape)
cosine_sim_df.head()

(32852, 32852)


title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Deep Hearts,The Morning After,House of Horrors,Shadow of the Blair Witch,The Burkittsville 7,Caged Heat 3000,Robin Hood,Betrayal,Satan Triumphant,Queerama
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.041569,0.008708,0.006937,0.005595,0.000000,0.006456,0.059202,0.000000,0.000000,...,0.0,0.051110,0.028298,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
Jumanji,0.041569,1.000000,0.000000,0.065065,0.000000,0.000000,0.000000,0.165721,0.028302,0.011462,...,0.0,0.000000,0.039299,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
Grumpier Old Men,0.008708,0.000000,1.000000,0.035846,0.010906,0.000000,0.033363,0.000000,0.000000,0.000000,...,0.0,0.099628,0.000000,0.0,0.0,0.0,0.106819,0.000000,0.0,0.0
Waiting to Exhale,0.006937,0.065065,0.035846,1.000000,0.093741,0.003806,0.063686,0.027484,0.000000,0.000000,...,0.0,0.135728,0.000000,0.0,0.0,0.0,0.121701,0.037622,0.0,0.0
Father of the Bride Part II,0.005595,0.000000,0.010906,0.093741,1.000000,0.000000,0.038016,0.000000,0.000000,0.000000,...,0.0,0.064015,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


# Content Based Recommend

### 유사도 데이터 중 가장 유사도 값이 큰 데이터순으로 k개만큼 가져옴

In [74]:
def genre_recommendations(target_title, matrix, items, k=10):
    recom_idx = matrix[target_title].values.argsort()[::-1][1:k+1]
    recom_title = items.iloc[recom_idx, :].title.values
    recom_genre = items.iloc[recom_idx, :].genres.values
    target_title_list = np.full(len(range(k)), target_title)
    target_genre_list = np.full(len(range(k)), items[items.title == target_title].genres.values)
    d = {
        'target_title':target_title_list,
        'target_genre':target_genre_list,
        'recom_title' : recom_title,
        'recom_genre' : recom_genre
    }
    return pd.DataFrame(d)

In [75]:
genre_recommendations('The Dark Knight Rises', cosine_sim_df, movie_data)

,target_title,target_genre,recom_title,recom_genre
0,The Dark Knight Rises,Action Crime Drama Thriller,The Dark Knight,Drama Action Crime Thriller
1,The Dark Knight Rises,Action Crime Drama Thriller,The Burglar,Crime Drama
2,The Dark Knight Rises,Action Crime Drama Thriller,Batman Begins,Action Crime Drama
3,The Dark Knight Rises,Action Crime Drama Thriller,Batman & Robin,Action Crime Fantasy
4,The Dark Knight Rises,Action Crime Drama Thriller,Batman,Fantasy Action
5,The Dark Knight Rises,Action Crime Drama Thriller,Raffles,Adventure Comedy Crime Drama History Romance T...
6,The Dark Knight Rises,Action Crime Drama Thriller,Hero at Large,Action Comedy Drama
7,The Dark Knight Rises,Action Crime Drama Thriller,DC Showcase: Catwoman,Action Adventure Animation Science Fiction
8,The Dark Knight Rises,Action Crime Drama Thriller,DC Super Hero Girls: Hero of the Year,Animation
9,The Dark Knight Rises,Action Crime Drama Thriller,Batman Returns,Action Fantasy
